# Visualize and measure the ellipticity of cucurbituril host/guest structures using ElliptiCBn

For more details, please see the documentation:

https://github.com/harmslab/ElliptiC

A manuscript describing this approach is forthcoming. 

In [16]:
#@title Set up environment.

#@markdown This cell configures the computing environment to run ElliptiC. Run 
#@markdown this cell before uploading an XYZ file to analyze. 

#@markdown To run the cell, click the "Play" button to the left.

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:
    %pip install git+https://github.com/harmsm/ElliptiC
    #%pip install -q ElliptiCBn

# ------------------------------------------------------------------------------
# Imports
    
import ElliptiCBn as ec

import numpy as np
import pandas as pd

import shutil
import os
import inspect

# Set these here in case the user does not run the cell below. 
bond_dist = 2.5
aspect_ratio_filter = 3 
oxygen_dist_cutoff = 2.9
min_num_carbons = 10 
max_num_carbons = 20
min_cycle_cc_bond_length = 1.3
max_cycle_cc_bond_length = 1.7 

In [17]:
#@title Set calculation parameters

bond_dist = 2.5 #@param
aspect_ratio_filter = 3 #@param
oxygen_dist_cutoff = 2.9 #@param
min_num_carbons = 10 #@param
max_num_carbons = 20 #@param
min_cycle_cc_bond_length = 1.3 #@param
max_cycle_cc_bond_length = 1.7 #@param


print("The default parameters should work for most calculations, but you can edit")
print("the following parameters if necessary to pull out the correct macrocycles.\n")
print("Parameter descriptions\n")
doc_string = dict(inspect.getmembers(ec.get_macrocycles))["__doc__"]
doc_string = doc_string.split("Returns")[0]

lines = doc_string.split("\n")

print("\n".join(lines[7:]))

The default parameters should work for most calculations, but you can edit
the following parameters if necessary to pull out the correct macrocycles.

Parameter descriptions

    bond_dist : float, default=2.5
        any atoms closer than bond distance (in angstroms) are identified as 
        part of a single molecule
    aspect_ratio_filter : float, default=3
        reject any identified cycles that have a PCA aspect ratio greater than
        aspect_ratio_filter. An aspect ratio of 1 corresponds to a square; an
        aspect ratio of 10 would be long and skinny. 
    oxygen_dist_cutoff : float, default=2.9
        when selecting the central cucurbituril macrocycle, identify carbons by
        removing any carbon closer than oxygen_dist_cutoff to an oxygen
    min_num_carbons : int, default=10
        reject any macrocycle with a central cycle that has less than 
        min_num_carbons
    max_num_carbons : int, default=10
        reject any macrocycle with a central cycle that h

In [ ]:
#@markdown Analyze a file. Press the "Play" button and upload an .xyz file.

out_base = "."
if RUNNING_IN_COLAB:
    
    from google.colab import files
    f = files.upload()
    filename = list(f.keys())[0]

    out_base = f"{filename}_out"
    if not os.path.isdir(out_base):
        shutil.mkdir(out_base)
    
atom_df = ec.ElliptiCBn.get_macrocycles(filename,
                                        bond_dist=bond_dist,
                                        aspect_ratio_filter=aspect_ratio_filter,
                                        oxygen_dist_cutoff=oxygen_dist_cutoff,
                                        min_num_carbons=min_num_carbons,
                                        max_num_carbons=max_num_carbons,
                                        min_cycle_cc_bond_length=min_cycle_cc_bond_length,
                                        max_cycle_cc_bond_length=max_cycle_cc_bond_length)
ellipticities, pca_vectors = ec.ElliptiCBn.get_ellipticity(atom_df)
ellipticities.to_excel(os.path.join(out_base,"results.xslx"))

fig = ec.ElliptiCBn.plot_results(atom_df,
                                 html_file=os.path.join(out_base,"results.html"),
                                 pca_vector_list=pca_vectors)
fig.show()



In [ ]:
#@markdown To download the results, press the "Play" button on the left. 

if RUNNING_IN_COLAB:

    shutil.make_archive(base_name=out_base,
                        format="zip",
                        base_dir=out_base)
    print(f"Put results in {out_base}.zip")
    
    files.download(f"{out_base}.zip")